In [1]:
import poppy
import jax
import jax.numpy as jnp
import functools
import galois
import numpy as np

In [2]:
p = 12421
n = 3
d = 20
R = jnp.arange(d)
f = poppy.field(p,n)
gf = galois.Field(p**n)
a = poppy.random((d,d),f,1)
ag = gf(np.array(a.proj()))

In [3]:
a.det().proj(), np.linalg.det(ag)

(Array([[1313593506912]], dtype=int64), GF(1313593506912, order=12421^3))

In [4]:
p = 17
n = 3
q = p ** n
f = poppy.field(p,n)
d = 100
a = poppy.random((d,d), f, poppy.SEED)
b = poppy.random((d,d), f, poppy.SEED)
c = poppy.random(1, f, poppy.SEED)

In [5]:
ai = a.inv()
bi = b.inv()
print(a.lift[0, :10])
(a-a@ai@a@bi@b).lift

[ 9  7 14  7 11  7  0  5  9 12]


Array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)